In [23]:
# Multimodal Rockfall Prediction System
# This notebook integrates both image and sensor models for rockfall risk prediction.

# --- Imports ---
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# --- Load Image Model ---
image_model_path = 'models/rockfall_model_improved.h5'
image_model = load_model(image_model_path, compile=False)

# --- Load Sensor Model (retrain for demo, or load if saved) ---
sensor_data = pd.read_csv('synthetic_rockfall_dataset (1).csv')
sensor_features = [
    'slope_angle', 'crack_density', 'displacement', 'strain',
    'pore_pressure', 'rainfall', 'temperature', 'vibration'
]
sensor_X = sensor_data[sensor_features]
sensor_y = (sensor_data['risk'] > 0.5).astype(int)
sensor_model = DecisionTreeClassifier(random_state=42, max_depth=3)
sensor_model.fit(sensor_X, sensor_y)

# --- Class mapping for image model ---
image_classes = ['low', 'medium', 'high']  # Adjust if your model uses a different order


In [24]:
# --- Predict from Image ---
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    preds = image_model.predict(x)
    class_idx = np.argmax(preds)
    return image_classes[class_idx], float(np.max(preds))


In [25]:
# --- Predict from Sensor Data ---
def predict_sensor(sensor_dict):
    df = pd.DataFrame([sensor_dict])
    proba = sensor_model.predict_proba(df)[:, 1][0]
    return round(proba * 100, 2)


In [26]:
# --- Example Usage ---
# Image prediction
img_path = 'data/test/high/download.jpg'  # Change to your test image path
img_class, img_conf = predict_image(img_path)
print(f"Image model prediction: {img_class} (confidence: {img_conf*100:.2f}%)")

# Sensor prediction
sensor_example = {
    'slope_angle': 45,
    'crack_density': 0.2,
    'displacement': 3,
    'strain': 0.007,
    'pore_pressure': 60,
    'rainfall': 40,
    'temperature': 22,
    'vibration': 0.4
}
sensor_proba = predict_sensor(sensor_example)
print(f"Sensor model predicted rockfall probability: {sensor_proba}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step
Image model prediction: medium (confidence: 40.08%)
Sensor model predicted rockfall probability: 0.0%


In [27]:
# --- Debug: Test model on a known row from training data ---
print("\n--- Debug: Model test on known training data row ---")
known_row = sensor_data.iloc[0][sensor_features].to_dict()
print("Known row input:", known_row)
known_proba = predict_sensor(known_row)
print(f"Predicted probability for known row: {known_proba}%")



--- Debug: Model test on known training data row ---
Known row input: {'slope_angle': 44.9816047538945, 'crack_density': 0.1851329288386196, 'displacement': 2.61705683735906, 'strain': 0.0067270299420889, 'pore_pressure': 57.19958782835928, 'rainfall': 39.363552027744205, 'temperature': 22.412847703891103, 'vibration': 0.3879945472888746}
Predicted probability for known row: 0.0%


In [29]:
# --- Extended Debug: Check model, data, and predictions ---
print("\n--- Extended Debug: Model and Data Checks ---")
print("sensor_y unique values:", sensor_y.unique())
print("sensor_y value counts:\n", sensor_y.value_counts())
print("\nTraining data feature stats:")
print(sensor_X.describe())

# Predict probabilities for all training data
train_probas = sensor_model.predict_proba(sensor_X)[:, 1]
print("\nFirst 10 predicted probabilities on training data:", train_probas[:10])
print("Min/Max/Mean predicted proba on training data:", train_probas.min(), train_probas.max(), train_probas.mean())

# Print feature importances
print("\nFeature importances:")
for feat, imp in zip(sensor_features, sensor_model.feature_importances_):
    print(f"{feat}: {imp:.4f}")

# Print DataFrame columns and dtypes for user input
print("\nUser input DataFrame columns and dtypes:")
user_df = pd.DataFrame([sensor_example])
print(user_df.dtypes)
print(user_df.columns)
print(user_df)



--- Extended Debug: Model and Data Checks ---
sensor_y unique values: [0 1]
sensor_y value counts:
 risk
0    1040
1     177
Name: count, dtype: int64

Training data feature stats:
       slope_angle  crack_density  displacement       strain  pore_pressure  \
count  1217.000000    1217.000000   1217.000000  1217.000000    1217.000000   
mean     48.706177       0.553689      5.091420     0.004864      46.878559   
std      12.020297       0.292914      2.893270     0.002844      29.222489   
min      29.273868      -0.016769     -0.084778    -0.000184      -2.789108   
25%      37.998747       0.259754      2.540848     0.002513      20.655184   
50%      47.378994       0.602371      4.955508     0.004908      42.100314   
75%      59.538985       0.818268      7.812888     0.007056      70.501901   
max      70.598147       1.011397     10.166404     0.010050     102.041584   

          rainfall  temperature    vibration  
count  1217.000000  1217.000000  1217.000000  
mean     48.

In [28]:
# --- User Input Section (for interactive use) ---
# Input for image path
img_path = input("Enter the image file path (e.g., data/test/high/download.jpg): ")
if img_path:
    img_class, img_conf = predict_image(img_path)
    print(f"Image model prediction: {img_class} (confidence: {img_conf*100:.2f}%)")
else:
    print("No image path provided.")

# Input for sensor data
print("\nEnter sensor data values:")
sensor_example = {}
for feature in sensor_features:
    val = input(f"Enter value for {feature}: ")
    try:
        val = float(val)
    except ValueError:
        print(f"Invalid input for {feature}, using 0.0 by default.")
        val = 0.0
    sensor_example[feature] = val
print("Sensor input DataFrame:")
print(pd.DataFrame([sensor_example]))
sensor_proba = predict_sensor(sensor_example)
print(f"Sensor model predicted rockfall probability: {sensor_proba}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Image model prediction: medium (confidence: 42.29%)

Enter sensor data values:
Sensor input DataFrame:
   slope_angle  crack_density  displacement  strain  pore_pressure  rainfall  \
0         45.0            2.0           4.0   0.004            0.5      50.0   

   temperature  vibration  
0         21.0        0.1  
Sensor model predicted rockfall probability: 0.0%
